# Preparing

In [ ]:
import numpy as np
from tqdm import tqdm

import tensorflow as tf
import keras
from keras.preprocessing import timeseries_dataset_from_array
from keras.callbacks import CSVLogger
from sklearn.metrics import classification_report

import modules.data as data
from modules.utilites import DataClass

In [ ]:
# Model parametrs
class Input_pars(DataClass):
    seq_len = 100


class CN_pars(DataClass):
    n_filters = 14
    dilation_steps = 4  # dilation = 2**dilation_step


class AN_pars(DataClass):
    attention_heads = 3
    blocks = 2
    share_weights = False


class FF_pars(DataClass):
    dropout_rate = 0.1


class Optimaser_pars(DataClass):
    lr = 0.0001
    adam_beta1 = 0.9
    adam_beta2 = 0.999


class Trainig_pars(DataClass):
    shuffle = True
    batch_size = 512
    epochs = 150


class Full_pars(DataClass):
    seq_len = 100
    cn = CN_pars()
    an = AN_pars()
    ff = FF_pars()
    optimazer = Optimaser_pars()
    training = Trainig_pars()


pars = Full_pars()

In [ ]:
# Load data
((x_train, y_train), (x_val, y_val), (x_test, y_test)) =(
# data.load_dataset(horizon=4)
data.load_saved_data()
)

In [ ]:
# # Save data
# data.save_data(name= 'train',x= x_train,y=y_train)
# data.save_data(name= 'val',x= x_val,y=y_val)
# data.save_data(name= 'test',x= x_test,y=y_test)

In [ ]:
# Dataset
ds_train = data.build_dataset(
    x=x_train,
    y=y_train,
    batch_size=pars.training.batch_size,
    seq_len=pars.seq_len,
)

ds_val = data.build_dataset(
    x=x_val,
    y=y_val,
    batch_size=pars.training.batch_size,
    seq_len=pars.seq_len,
)

ds_test = data.build_dataset(
    x=x_test,
    y=y_test,
    batch_size=pars.training.batch_size,
    seq_len=pars.seq_len,
)

print(
    f'Train x: {str(x_train.shape): <15} - y: {str(y_train.shape): <15}'+\
    f' - tensor: {len(ds_train)} - {list(ds_train.element_spec[0].shape)}',
    f'Val   x: {str(x_val.shape): <15} - y: {str(y_val.shape): <15}'+\
    f' - tensor: {len(ds_val)} - {list(ds_val.element_spec[0].shape)}',
    f'Test  x: {str(x_test.shape): <15} - y: {str(y_test.shape): <15}'+\
    f' - tensor: {len(ds_test)} - {list(ds_test.element_spec[0].shape)}',
    sep='\n',
)

# Test basic model

In [ ]:
# # Model
# inputs = blocks.input_block(pars.seq_len)
# x= inputs


# x = blocks.cnn_block(
#     input_layer=x,
#     filters=pars.cn.n_filters,
#     dilation_steps=pars.cn.dilation_steps,
# )

# # x= keras.layers.Reshape(target_shape=(100,14,1))(x)
# # x = LobFeatures.lob_inception(x)
# # x= LobFeatures.lob_cnn(x)


# x = blocks.norm_block(input_layer=x)

# x = blocks.positional_encoder_block(input_layer=x)


# x = blocks.transformer_block(
#     input_layer=x,
#     n_blocks=pars.an.blocks,
#     n_heads=pars.an.attention_heads,
#     share_weights=pars.an.share_weights,
# )


# x = blocks.ffn_block(
#     input_layer=x,
#     dropout_rate=pars.ff.dropout_rate,
# )


# model = keras.Model(inputs=inputs, outputs=x)
# model.summary(
#     line_length=100,
#     expand_nested=True,
# )

In [ ]:
# # CompiLe
# model.compile(
#     keras.optimizers.Adam(
#         learning_rate=pars.optimazer.lr,
#         beta_1=pars.optimazer.adam_beta1,
#         beta_2=pars.optimazer.adam_beta2,
#         name="Adam",
#     ),
#     loss=keras.losses.SparseCategoricalCrossentropy(),
#     metrics=[
#         keras.metrics.CategoricalAccuracy(name='accurancy'),
#         keras.metrics.SparseCategoricalAccuracy(name='sparce_accurancy'),
#     ],
# )

In [ ]:
# # Train
# model.fit(
#     ds_train,
#     # epochs=pars.training.epochs,
#     epochs=10,
#     validation_data=ds_val,
#     # callbacks=[
#     #     CSVLogger(r'logs/model1/log.csv', append=True, separator=';')
#     # ]
# )

# Search

In [ ]:
from keras_tuner.tuners import GridSearch
import keras_tuner

In [ ]:
import models.base_model

In [ ]:
models.base_model

In [ ]:
hp =keras_tuner.HyperParameters()
cn_layers=hp.Int(
    'cn_layers',
    min_value=3,
    max_value=5,
    step=1,
)
cn_filters=hp.Int(
    'cn_filters',
    min_value=14,
    max_value=16,
    step=1,
)

In [ ]:
def test_model(hp):
    cn_layers=hp.Int(
    'cn_layers',
    min_value=3,
    max_value=5,
    step=1,
    )
    cn_filters=hp.Int(
        'cn_filters',
        min_value=14,
        max_value=16,
        step=1,
    )
    
    return base_model.build_model(
        optimizer=keras.optimizers.Adam(
            learning_rate=0.1,
            beta_1=0.99,
            beta_2=0.9,
            name="Adam",
        ),
        cn__n_filters=cn_filters,
        cn__dilation_steps=cn_layers,
    )


In [ ]:
tuner = keras_tuner.RandomSearch(
    hypermodel=test_model,
    objective="val_accuracy",
    max_trials=3,
    executions_per_trial=2,
    directory="my_dir",
    project_name="helloworld",
)

In [ ]:
tuner.search(ds_train, epochs=2,)